In [209]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from math import trunc

In [210]:
k = 3
sigK = 1000
N = 100
sigY = 5
p = np.array([0.4, 0.3, 0.3])
Bs = np.array(([10, 5, 2],[-0.0100, -0.1000, -0.5000]))

In [211]:
s = np.array([])
sigmaK = np.array([[0,0,0]]).T
for i in range(k):
    s = np.ones((k,1))
    s[i] = s[i] + sigK
    sigmaK = np.concatenate((sigmaK, s), axis = 1)

# temp line since np.concatenate needs a dummy column to work 
sigmaK = np.delete(sigmaK, 0, axis = 1)


In [212]:
def discreteinrnd(p, m, n):
    X = np.zeros((m,n))
    for i in range(m*n):
        u = np.random.rand()
        j = np.cumsum(p)
        I = np.argwhere(u < j)
        X [i] = min(I)
    return X
        

In [213]:
def WATsim(N, K, C, sigmaK, p):
    clas = np.array([])
    W = np.array([[0,0,0]]).T
    
    for i in range(N):
        clas = np.append(clas, discreteinrnd(p, 1, 1))
        
        temp2 = sigmaK[trunc(clas[i])]
        temp2 = np.append(temp2, np.array([1, 1, 1, 1, 1, 1]))
        temp2 = temp2.reshape(3,3)                
        
        temp = abs(np.random.multivariate_normal(np.zeros(C), temp2))
        temp = np.true_divide(temp, np.sum(temp))
        W = np.concatenate((W, temp.reshape(3,1)), axis = 1)        
    
    return clas, W, W.T @ W


In [214]:
def MCsim(W,T, B, sigmaE):
    N, K = W.T.shape
    
    Y = np.array([])
    
    for i in range(1,N):
        e = np.random.normal(0, sigmaE, (25,1))
        k = W.T[i-1] @ B.T @ T[i-1] + e.T
        Y = np.append(Y, k).reshape(i,25)
    return Y
        


In [215]:
clas,W,A = WATsim(N, k, k, sigmaK.T, p)

In [216]:
Tall = np.array(np.ones((1,25)))
t = np.array([i for i in range(1,26)])
Tall = np.append(Tall, t).reshape(2,25)
Tall = np.append(Tall,Tall).reshape(2,2,25)

t2 = np.append(np.ones((1,25)), t).reshape(2,25)

for i in range(3,N+1):
    Tall = np.append(Tall, t2).reshape(i, 2, 25)


In [217]:
S = MCsim(W, Tall, Bs, 0.1)

